# オブジェクト検出のライブデモプログラムの作り方を理解する  

ここでは、jupyter notebookを使用したライブデモの作り方を学びます。  
ムービーファイルやWebCam (USBカメラ)から画像を読み込み、リアルタイムに推論結果を表示するデモの構成を見ていきましょう。

## 必要なライブラリのインポート  

Note: Windows上でOpenCVでWebCamをオープンすると30秒以上待たされることがあります。これを回避するため、`OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS`環境変数に`0`を設定しています。これは`import cv2`よりも先に記述する必要があります。

In [ ]:
import os, sys
from io import BytesIO
import time

os.environ["OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS"] = "0"   # Expedite WebCam initialization in OpenCV (on Windows)
import cv2
import numpy as np

import openvino as ov

import IPython

## 表示用のクラスラベル

In [ ]:
coco_class_label = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
                    'fire hydrant', 'street_sign', 'stop sign','parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 
                    'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 
                    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
                    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
                    'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
                    'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet',
                    'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                    'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush',
                    'hair brush']

coco_class_label = [ '_background_' ] + coco_class_label

## TensorFlowの物体検出モデル (ssd-mobilenet-v2)を読み込み、OpenVINO IR形式に変換し、ファイルに保存する。

In [ ]:
import tensorflow_hub

if not os.path.exists('ssd-mobilenet-v2.xml'):
    detector = tensorflow_hub.load("https://www.kaggle.com/models/tensorflow/ssd-mobilenet-v2/frameworks/TensorFlow2/variations/ssd-mobilenet-v2/versions/1")
    ov_model = ov.convert_model(detector)
    ov.save_model(ov_model, 'ssd-mobilenet-v2.xml')

## 保存したOpenVINO IRモデルを読み込む

In [ ]:
ov_core = ov.Core()
ov_model = ov_core.read_model('ssd-mobilenet-v2.xml')
ov_model.reshape((1,300,300,3))
compiled_model = ov.compile_model(ov_model, 'CPU', config={'CACHE_DIR':'./cache'})
#compiled_model = ov.compile_model('mobilenet_v2.xml', 'GPU.0')

## ビデオストリームをオープンする  

`cv2.VideoCapture()`の引数に`0`を渡すと0番目のUSBカメラを、ファイル名を指定するとムービーファイルをオープンします。

In [ ]:
video_source = 0                          # 0, "sample.mp4", etc  (0 means Webcam #0)
#video_source = "people-detection.mp4"

cap = cv2.VideoCapture(video_source)
if cap.isOpened():
    cap.set(cv2.CAP_PROP_FRAME_WIDTH , 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
else:
    print(f'Failed to open {video_source}.') 

## 表示部分 (画像読み込み＋推論実行＋後処理＋結果描画）

`ssd-mobilenet-v2`はYolo系とは異なる、SSD (Single Shot multibox Detector)系のアーキテクチャのモデルです。SSDモデルのバックボーン部にmobilenetアーキテクチャのモデルを利用しているモデルです。  
Yolo系とは出力テンソルのフォーマットが異なり、通常はNMS済みのBboxデータだけが出力されます。つまり、モデル側でNMS処理を行ってくれているので、ユーザープログラム側ではNMSを実行する必要はありません。Bbox信頼度に基づき、推論結果を画像上に描画すればOKです。  

このセルでは一定時間ループ処理を行ったのち終了します。ループ内ではユーザーからのキーボードやマウスからの入力を受け取る部分がなく、途中で中断するにはJupyter notebookのカーネルをinterruptするしかありません。  

In [ ]:
# 描画した画像を表示する準備
notebook_display = IPython.display.display(display_id=1)

stime = time.time()

while time.time() - stime < 50:                               # 指定の秒数だけ実行する (50秒)

    # ビデオストリームから１フレーム読み込む
    ret, img = cap.read()                                     
    if ret is False:
        break

    # プリプロセス
    img = cv2.resize(img, (300, 300))
    tensor = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)             # BGR -> RGB
    tensor = np.expand_dims(tensor, axis=0).astype(np.uint8)  # HWC -> NHWC (h,w,c) -> (1,h,w,c)
    #tensor = (tensor.astype(np.float32)/255.0-0.5)/0.2

    # 推論実行
    res = compiled_model.infer_new_request(tensor)

    img_w, img_h, _ = img.shape                               # 入力画像のサイズを取得

    # ポストプロセス ＋ 結果の画面描画
    num_detections = int(res['num_detections'][0])            # 検出ボックス数
    for (rel_y0,rel_x0,rel_y1,rel_x1), class_id, score in zip(res['detection_boxes'][0], res['detection_classes'][0], res['detection_scores'][0]):  # Bboxごとに処理
        if score > 0.65:                 # Bboxの信頼度が0.65 (65%)以上ならBboxを描画。それ以下なら無視。
            x0 = int(img_w * rel_x0)
            y0 = int(img_h * rel_y0)
            x1 = int(img_w * rel_x1)
            y1 = int(img_h * rel_y1)
            # Bboxの描画
            cv2.rectangle(img, (x0, y0), (x1, y1), (0,255,0), 1)
            # クラスラベルの描画
            text = coco_class_label[int(class_id)]
            (w, h), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, fontScale=1, thickness=1)
            cv2.rectangle(img, (x0, y0), (x0 + w, y0 - h - baseline), color=(0,255,0), thickness=-1)
            cv2.putText(img, text, (x0, y0 - baseline), cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(0,0,0), thickness=1)

    # 画像を表示
    _, jpg_img = cv2.imencode('.jpg', img, [cv2.IMWRITE_JPEG_QUALITY, 90])
    notebook_display.update(IPython.display.Image(data=BytesIO(jpg_img).getvalue()))

print('Finished.')

## ビデオストリームを閉じる

In [ ]:
cap.release()

## まとめ  

ここではJupyter notebookを利用してリアルタイムデモを作るしくみの基礎を学びました。Jupyter bookにはほかにも画像を表示するための仕組みなどがありますので調べてみてください。  

------------------
## おまけ  

普通にOpenCVの`cv2.imshow()`を使って結果を表示することも可能です。その場合、Jupyter notebookとは別のWindowが開き、そこに結果が表示されます。

In [13]:
video_source = 0                          # 0, "sample.mp4", etc  (0 means Webcam #0)
#video_source = "people-detection.mp4"

cap = cv2.VideoCapture(video_source)
if cap.isOpened():
    cap.set(cv2.CAP_PROP_FRAME_WIDTH , 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
else:
    print(f'Failed to open {video_source}.') 

stime = time.time()

key = -1
while key!=27 and key!=ord('q'):                               # ESCか'q'が押されるまで実行

    # ビデオストリームから１フレーム読み込む
    ret, img = cap.read()                                     
    if ret is False:
        break

    # プリプロセス
    img = cv2.resize(img, (300, 300))
    tensor = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)             # BGR -> RGB
    tensor = np.expand_dims(tensor, axis=0).astype(np.uint8)  # HWC -> NHWC (h,w,c) -> (1,h,w,c)
    #tensor = (tensor.astype(np.float32)/255.0-0.5)/0.2

    # 推論実行
    res = compiled_model.infer_new_request(tensor)

    img_w, img_h, _ = img.shape                               # 入力画像のサイズを取得

    # ポストプロセス ＋ 結果の画面描画
    num_detections = int(res['num_detections'][0])            # 検出ボックス数
    for (rel_y0,rel_x0,rel_y1,rel_x1), class_id, score in zip(res['detection_boxes'][0], res['detection_classes'][0], res['detection_scores'][0]):  # Bboxごとに処理
        if score > 0.65:                 # Bboxの信頼度が0.65 (65%)以上ならBboxを描画。それ以下なら無視。
            x0 = int(img_w * rel_x0)
            y0 = int(img_h * rel_y0)
            x1 = int(img_w * rel_x1)
            y1 = int(img_h * rel_y1)
            # Bboxの描画
            cv2.rectangle(img, (x0, y0), (x1, y1), (0,255,0), 1)
            # クラスラベルの描画
            text = coco_class_label[int(class_id)]
            (w, h), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, fontScale=1, thickness=1)
            cv2.rectangle(img, (x0, y0), (x0 + w, y0 - h - baseline), color=(0,255,0), thickness=-1)
            cv2.putText(img, text, (x0, y0 - baseline), cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(0,0,0), thickness=1)

    # 画像を表示
    cv2.imshow('Result', img)          # OpenCVを使って結果画像を表示 (別ウインドウに表示される)
    key = cv2.waitKey(1)               # キー入力を読み取る (OpenCVでは`imshow()`を読んだだけでは画像は表示されません。`waitKey()`が呼ばれたタイミングでウインドウへの実際の描画がなされるので、`waitKey()`を省略することはできません。

cap.release()                   # ビデオストリームを閉じる
cv2.destroyAllWindows()         # すべてのOpenCVウインドウを閉じる

print('Finished.')

Finished.
